**Kütüphanelerin Yüklenmesi ve Cihaz Seçimi**

In [ ]:
# --- Temel kütüphaneler ---
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

# --- GPU veya CPU seçimi ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Kullanılan cihaz: {device}")

**Parametreler ve Veri Yolları**

In [ ]:
# --- Görüntü boyutu, batch size ve epoch sayısı ---
height = 180
width = 180
batch_size = 32
epochs_size = 10

# --- Veri yolları (Kaggle veya local path) ---
train_path = "/kaggle/input/vehicle-classification/train"
val_path = "/kaggle/input/vehicle-classification/val"
test_path = "/kaggle/input/vehicle-classification/test"

**Veri Dönüşümleri (Data Augmentation)**

In [ ]:
# --- Eğitim verisi için dönüşümler ---
transform_train = transforms.Compose([
    transforms.Resize((height, width)),
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, hue=0.1),
    transforms.ToTensor(),
])

# --- Doğrulama / test verisi için dönüşümler ---
transform_val = transforms.Compose([
    transforms.Resize((height, width)),
    transforms.ToTensor(),
])

**Dataset ve DataLoader Oluşturma**

In [ ]:
# --- Dataset ---
data_train = ImageFolder(root=train_path, transform=transform_train)
data_val = ImageFolder(root=val_path, transform=transform_val)

# --- DataLoader ---
train_loader = DataLoader(data_train, batch_size=batch_size, shuffle=True, num_workers=2)
val_loader = DataLoader(data_val, batch_size=batch_size, shuffle=True, num_workers=2)

# --- Sınıf bilgisi ---
category = data_train.classes
num_classes = len(category)
print(f"Sınıflar: {category}")
print(f"Sınıf sayısı: {num_classes}")


**Görselleştirme (Örnek Eğitim Görüntüleri)**

In [ ]:
plt.figure(figsize=(10, 10))
images, labels = next(iter(train_loader))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    img = images[i].permute(1, 2, 0).numpy()  # [C,H,W] -> [H,W,C]
    plt.imshow(img)
    plt.title(category[labels[i]])
    plt.axis('off')
plt.tight_layout()
plt.show()


**Model Tanımlama (CNN)**

In [ ]:
class VehicleClassifier(nn.Module):
    def __init__(self, num_classes=10):
        super(VehicleClassifier, self).__init__()
        
        # --- Convolutional Block 1 ---
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=0)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2)
        self.drop1 = nn.Dropout(0.1)
        
        # --- Convolutional Block 2 ---
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2)
        self.drop2 = nn.Dropout(0.1)
        
        # --- Convolutional Block 3 ---
        self.conv3 = nn.Conv2d(64, 128, 3)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(2)
        self.drop3 = nn.Dropout(0.2)
        
        # --- Convolutional Block 4 ---
        self.conv4 = nn.Conv2d(128, 256, 3)
        self.bn4 = nn.BatchNorm2d(256)
        self.pool4 = nn.MaxPool2d(2)
        self.drop4 = nn.Dropout(0.2)
        
        # --- Fully Connected Layers ---
        self.fc1 = nn.Linear(256*9*9, 512)
        self.bn5 = nn.BatchNorm1d(512)
        self.drop5 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, num_classes)
        
        self.relu = nn.ReLU()
    
    def forward(self, x):
        # --- Conv Block 1 ---
        x = self.pool1(self.relu(self.bn1(self.conv1(x))))
        x = self.drop1(x)
        
        # --- Conv Block 2 ---
        x = self.pool2(self.relu(self.bn2(self.conv2(x))))
        x = self.drop2(x)
        
        # --- Conv Block 3 ---
        x = self.pool3(self.relu(self.bn3(self.conv3(x))))
        x = self.drop3(x)
        
        # --- Conv Block 4 ---
        x = self.pool4(self.relu(self.bn4(self.conv4(x))))
        x = self.drop4(x)
        
        # --- Flatten ---
        x = x.view(x.size(0), -1)
        
        # --- Fully Connected Layers ---
        x = self.drop5(self.relu(self.bn5(self.fc1(x))))
        x = self.fc2(x)
        
        return x

# --- Model oluştur ---
model = VehicleClassifier(num_classes=num_classes).to(device)


**Loss ve Optimizer**

In [ ]:
criterion = nn.CrossEntropyLoss()  # Kategorik sınıflar için
optimizer = optim.AdamW(model.parameters())

# --- Model özeti ---
print(model)
print(f"\nToplam parametre sayısı: {sum(p.numel() for p in model.parameters()):,}")


**Eğitim ve Doğrulama Fonksiyonları**

In [ ]:
# --- Eğitim fonksiyonu ---
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss, correct, total = 0, 0, 0
    
    for images, labels in tqdm(loader, desc="Training"):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    
    return running_loss / len(loader), 100.*correct/total

# --- Doğrulama fonksiyonu ---
def validate(model, loader, criterion, device):
    model.eval()
    running_loss, correct, total = 0, 0, 0
    
    with torch.no_grad():
        for images, labels in tqdm(loader, desc="Validation"):
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    
    return running_loss / len(loader), 100.*correct/total


**Eğitim Döngüsü**

In [ ]:
history = {'loss': [], 'accuracy': [], 'val_loss': [], 'val_accuracy': []}

for epoch in range(epochs_size):
    print(f"\nEpoch {epoch+1}/{epochs_size}")
    
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    val_loss, val_acc = validate(model, val_loader, criterion, device)
    
    history['loss'].append(train_loss)
    history['accuracy'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_accuracy'].append(val_acc)
    
    print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")

print("\n✅ Eğitim tamamlandı!")


**Eğitim Grafikleri**

In [ ]:
epochs_range = range(epochs_size)

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, history['accuracy'], label='Training Accuracy')
plt.plot(epochs_range, history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.title('Model Accuracy')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(epochs_range, history['loss'], label='Training Loss')
plt.plot(epochs_range, history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Model Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()


**Model Kaydetme**

In [ ]:
torch.save(model.state_dict(), 'vehicle_classifier.pth')
print("✅ Model kaydedildi: vehicle_classifier.pth")
